In [ ]:
import os
import cv2
import rasterio
from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, Polygon
from shapely.ops import cascaded_union
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from geopandas import GeoSeries
from shapely.geometry import Polygon
from rasterio.windows import Window
from rasterio.plot import reshape_as_image
# import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

%matplotlib inline

In [ ]:
PROJECT_DIR = '/home/ymi/data/ucu_data'

RASTER_PATH = os.path.join(PROJECT_DIR, 'T34JEP_20170101T082332/T34JEP_20170101T082332_TCI.jp2')
TRAIN_POLYGONS_PATH = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/train/train.shp')
TEST_POLYGONS_PATH = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/test/test.shp')
TRAIN_POLYGONS_CONVERTED = os.path.join(PROJECT_DIR, 'train-20220726T194123Z-001/train/train.geojson')
RASTER_MASK_PATH = os.path.join(PROJECT_DIR, 'mask.jp2')
DROP_CSV_PATH = os.path.join(PROJECT_DIR, 'drop.csv')
FRAGMENT_STORAGE = os.path.join(PROJECT_DIR, 'split')
CROPPED_IMAGES = os.path.join(PROJECT_DIR, 'images_cropped_rgb')

Reading Raster with rasterio

In [ ]:
# read dropped csv 
drop_df = pd.read_csv(DROP_CSV_PATH)
drop_list = drop_df['images_to_drop'].str.split("/").str[-1].tolist()

In [ ]:
# get num of dropped masks

In [ ]:
# masks and images paths
imgs_path = os.path.join(FRAGMENT_STORAGE, 'images')
masks_path = os.path.join(FRAGMENT_STORAGE, 'masks')

# only used masks 
mask_names = os.listdir(masks_path)
used_masks = [mask_name for mask_name in mask_names if mask_name not in drop_list]

# only used images
used_image_names = [os.path.join(imgs_path, mask_name.replace( "mask_", "T34JEP_20170101T082332_TCI_")) for mask_name in used_masks]
used_masks = [os.path.join(masks_path, img_path) for img_path in used_masks]


Unet architecture

![title](unet.png)

In [ ]:
def build_model(input_shape):
    'build unet model from scratch'
    inputs = Input(input_shape)

    
    outputs = Conv2D(1, (1,1), activation = 'sigmoid')(conv9)
    
    model = Model(inputs=[inputs], outputs=[outputs])
    
    
    return model



In [ ]:
# add early stopping and model save
early_stopping = EarlyStopping(patience=5, verbose=1)
model_checkpoint = ModelCheckpoint("model4.hdf5", save_best_only=True, verbose=1, monitor='dice_coef', mode='max')

In [ ]:
# read image and mask
def load_image(img_path):
      
    img = cv2.imread(img_path)

    return img

def load_mask(img_path):
      
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img[img>1]=1
        
    return img

In [ ]:
#generate X
def generate_X(img_list):
    X = np.empty((len(img_list), 256, 256, 3))

    for i, item in enumerate(img_list):
        img = load_image(item)
        X[i,] = img

    return X



In [ ]:
#generate Y
def generate_y(mask_list):
    y = np.empty((len(mask_list), 256, 256, 1), dtype=int)

    for i, item in enumerate(mask_list):
        mask = load_mask(item)
        y[i, :, : , 0] = mask
    
    
    return y.astype(np.float32)

In [ ]:
# loss and metrics
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score

def bce_dice_loss(y_true, y_pred):
    return 0.2 * binary_crossentropy(y_true, y_pred) + 0.8 * dice_loss(y_true, y_pred)

In [ ]:
# generate X


In [ ]:
# generate Y


In [ ]:
# build model with (256, 256, 3) input 


In [ ]:
# check model summary

In [ ]:
# compile model
model.compile(optimizer=Adam(learning_rate = 1e-4),
              loss=bce_dice_loss,
              metrics=[dice_coef])

In [ ]:
# run train
model.fit(X_train,
          y_train,
          batch_size=8, 
          epochs=25,
          verbose=1,
          callbacks=[early_stopping,
                     model_checkpoint])

In [ ]:
# load model from disk need to add custom objects bce_dice_loss and dice_coef

In [ ]:
# take some random image from training 
z = X_train[0].reshape((1,256,256,3))

# predict 

# binarize prediction


# plot the results with matplotlib
f, axarr = plt.subplots(1, 3)
f.set_size_inches(15, 15)
axarr[0].imshow(y_train[0].reshape((256,256)))
axarr[1].imshow(test.reshape((256,256)))
axarr[2].imshow(X_train[0].astype(np.uint8))